# Topic Modeling - BERTopic

In [1]:
# from datasets import load_dataset
from bertopic import BERTopic
from transformers import AutoModel, AutoTokenizer
from sklearn.feature_extraction.text import CountVectorizer
import torch
import pandas as pd

from tqdm import tqdm
# from gpt4all import GPT4All
from openai import OpenAI
import json
import re

# Load Dialog Data

In [2]:
df = pd.read_parquet('input_data/ccd_unlabeled_final.parquet') 
df.head()

date              dlg_id  \
0  2024-10-05  2024-10-07_1017--0   
1  2024-10-05  2024-10-07_1017--1   
2  2024-10-05  2024-10-07_1017--2   
3  2024-10-05  2024-10-07_1017--3   
4  2024-10-05  2024-10-07_1017--4   

                                                text  
0                      i am still waiting on my card  
1  what can i do if my card still hasnt arrived a...  
2  i have been waiting over a week is the card st...  
3  can i track my card while it is in the process...  
4  how do i know if i will get my card or if it i...

In [3]:
# convert to list
ds_sgd_text = df['text'].tolist()
for i in range(5):
    print(ds_sgd_text[i])

i am still waiting on my card
what can i do if my card still hasnt arrived after 2 weeks
i have been waiting over a week is the card still coming
can i track my card while it is in the process of delivery
how do i know if i will get my card or if it is lost


# Load Huggingface Transformer Model and Tokenizer

In [4]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"  # You can change this model to any other transformer
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

# Create Embeddings

In [5]:
def embed_texts(texts, tokenizer, model):
    # Tokenize and encode the texts
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt", max_length=512)
    
    # Get the model embeddings
    with torch.no_grad():
        model_output = model(**inputs)
    
    # Use the CLS token representation as the sentence embedding
    embeddings = model_output.last_hidden_state[:, 0, :].numpy()
    return embeddings

# Generate embeddings for the dialogues
embeddings = embed_texts(ds_sgd_text, tokenizer, model)

# Fit BERTopic using embeddings

In [6]:
# BERTopic needs a CountVectorizer for creating topics
vectorizer_model = CountVectorizer(stop_words="english", ngram_range=(1, 2))

# Initialize BERTopic model
topic_model = BERTopic(vectorizer_model=vectorizer_model)

# Fit the model to the dialogues and embeddings
topics, probabilities = topic_model.fit_transform(ds_sgd_text, embeddings)

# Explore the Topics

In [7]:
topic_info = topic_model.get_topic_info()
topic_info.shape

(319, 5)

In [8]:
# Display the most frequent topics
topic_info.head(5)  # top N by count

Topic  Count                                           Name  \
0     -1   2109  -1_credit_credit card_savings_savings account   
1      0    559                       0_thank_thats_thanks_ill   
2      1    410    1_bank_current account_current_account bank   
3      2    264        2_alex_thank alex_help alex_thanks alex   
4      3    187      3_debit card_debit_new debit_received new   

                                      Representation  \
0  [credit, credit card, savings, savings account...   
1  [thank, thats, thanks, ill, alright, thank hel...   
2  [bank, current account, current, account bank,...   
3  [alex, thank alex, help alex, thanks alex, tha...   
4  [debit card, debit, new debit, received new, h...   

                                 Representative_Docs  
0  [Hi Lisa! I'm interested in opening a forex ac...  
1  [No, that's all. Thank you for your help., No,...  
2  [That's perfect, Alex. Thank you so much for y...  
3  [No, that’s all for now. Thank you, Alex., Tha...  
4  [Hi, I recently received a new debit card from...

In [19]:
# Examine the top words for each topic in the Top N topics with p-values
for topic_idx in topic_info['Topic'][:5]:
    if topic_idx != -1:  # Ignore the -1 label which is for outliers
        print(f"\nTopic {topic_idx}:")
        print(topic_model.get_topic(topic_idx))


Topic 0:
[('thank', 0.011744725860867257), ('thats', 0.010876610132390253), ('thanks', 0.009982222141130061), ('ill', 0.009345783225502065), ('alright', 0.00874297305686414), ('thank help', 0.008257291481334358), ('appreciate', 0.007933931105012353), ('yes', 0.0072986662841044115), ('sounds', 0.006827889746731692), ('thanks help', 0.0066705871254980195)]

Topic 1:
[('bank', 0.012666325720750321), ('current account', 0.00933434677811194), ('current', 0.00781291129594798), ('account bank', 0.007588052852946665), ('banking', 0.006669192550151189), ('service', 0.006212568901258003), ('experience', 0.005635888250314769), ('considering', 0.005158539562635057), ('alex', 0.004978372153505992), ('services', 0.0049641364957664635)]

Topic 2:
[('alex', 0.03075987141870955), ('thank alex', 0.029436540796896702), ('help alex', 0.025356603390048457), ('thanks alex', 0.020707369954840987), ('thanks', 0.018025349958951532), ('thank', 0.01801193894933885), ('alex thank', 0.01630066140507997), ('day', 

# Use LLM to Generate Most Likely Topic
- using LLM - OpenAI

### Select N Topics first, we don't want the LLM to generate for thousands of topics

In [37]:
top_N = 20
topics_df = topic_info.loc[1: top_N, :].copy()  # topic at row 0 is for outliers, we start from 1:
topics_df.reset_index(drop=True, inplace=True)
print(topics_df.shape)
display(topics_df.head(5))

(20, 5)


Topic  Count                                         Name  \
0      0    559                     0_thank_thats_thanks_ill   
1      1    410  1_bank_current account_current_account bank   
2      2    264      2_alex_thank alex_help alex_thanks alex   
3      3    187    3_debit card_debit_new debit_received new   
4      4    162   4_fixed deposit_fixed_deposit_deposit bank   

                                      Representation  \
0  [thank, thats, thanks, ill, alright, thank hel...   
1  [bank, current account, current, account bank,...   
2  [alex, thank alex, help alex, thanks alex, tha...   
3  [debit card, debit, new debit, received new, h...   
4  [fixed deposit, fixed, deposit, deposit bank, ...   

                                 Representative_Docs  
0  [No, that's all. Thank you for your help., No,...  
1  [That's perfect, Alex. Thank you so much for y...  
2  [No, that’s all for now. Thank you, Alex., Tha...  
3  [Hi, I recently received a new debit card from...  
4  [Hi, I’m interested in opening a fixed deposit...

In [38]:
from pathlib import Path
from dotenv import load_dotenv
import os

cwd = Path.cwd()
env_path = cwd.parent.parent.parent.parent
print(env_path)

load_dotenv(dotenv_path=env_path / '.env')
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key: Found")
else:
    print("API Key not found. Please ensure it is in the .env file.")

client = OpenAI()

D:\OneDrive\RChai\Documents\NUS_ISS_Prjs\4_PLP
OpenAI API Key: Found


In [39]:
def make_prompt(rep_words: list, rep_docs: list):
    p1 = f"""The representative words are: {rep_words}. """
    p2 = f"""The representative docs are: {rep_docs}. """
    p3 = """From the list of Representative words and list of Representative Docs, create a topic and a short explanation. """
    p4 = """Please reply using in JSON format using the following keys: 'topic', explanation'. Do not reply with anything else.  """
    p5 = """ """
    p6 = """ """
    return p1 + p2 + p3 + p4 + p5 + p6  

# make_prompt(["apple, orange, banana"], ['a', 'b', 'c'])

In [40]:
def get_dict_from_openai_completion(c):
    c_content = c.choices[0].message.content
    match = re.search(r'\{.*?\}', c_content, re.DOTALL)
    if match:
        json_str = match.group(0)
        return json.loads(json_str)
    else:
        print("No match found")
        print(c)
        print('*' * 50)

In [41]:
topics_df.head()

Topic  Count                                         Name  \
0      0    559                     0_thank_thats_thanks_ill   
1      1    410  1_bank_current account_current_account bank   
2      2    264      2_alex_thank alex_help alex_thanks alex   
3      3    187    3_debit card_debit_new debit_received new   
4      4    162   4_fixed deposit_fixed_deposit_deposit bank   

                                      Representation  \
0  [thank, thats, thanks, ill, alright, thank hel...   
1  [bank, current account, current, account bank,...   
2  [alex, thank alex, help alex, thanks alex, tha...   
3  [debit card, debit, new debit, received new, h...   
4  [fixed deposit, fixed, deposit, deposit bank, ...   

                                 Representative_Docs  
0  [No, that's all. Thank you for your help., No,...  
1  [That's perfect, Alex. Thank you so much for y...  
2  [No, that’s all for now. Thank you, Alex., Tha...  
3  [Hi, I recently received a new debit card from...  
4  [Hi, I’m interested in opening a fixed deposit...

In [51]:
completion_lst = []
import time

for i in tqdm(range(topics_df.shape[0])):
    prompt = make_prompt(topics_df.loc[i, 'Representation'], topics_df.loc[i, 'Representative_Docs'])
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    completion_lst.append(get_dict_from_openai_completion(completion))
    time.sleep(1)
    
print('\n\nall tasks completed\n\n')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:55<00:00,  2.76s/it]



all tasks completed




In [52]:
topic_lst = [i['topic'] for i in completion_lst]
explain_lst = [i['explanation'] for i in completion_lst]

topics_df['topic'] = topic_lst
topics_df['explanation'] = explain_lst
topics_df.head()

Topic  Count                                         Name  \
0      0    559                     0_thank_thats_thanks_ill   
1      1    410  1_bank_current account_current_account bank   
2      2    264      2_alex_thank alex_help alex_thanks alex   
3      3    187    3_debit card_debit_new debit_received new   
4      4    162   4_fixed deposit_fixed_deposit_deposit bank   

                                      Representation  \
0  [thank, thats, thanks, ill, alright, thank hel...   
1  [bank, current account, current, account bank,...   
2  [alex, thank alex, help alex, thanks alex, tha...   
3  [debit card, debit, new debit, received new, h...   
4  [fixed deposit, fixed, deposit, deposit bank, ...   

                                 Representative_Docs  \
0  [No, that's all. Thank you for your help., No,...   
1  [That's perfect, Alex. Thank you so much for y...   
2  [No, that’s all for now. Thank you, Alex., Tha...   
3  [Hi, I recently received a new debit card from...   
4  [Hi, I’m interested in opening a fixed deposit...   

                                               topic  \
0                         Gratitude and Confirmation   
1  Customer Satisfaction and Account Services in ...   
2    Expressing Gratitude and Acknowledgment to Alex   
3                          Activating New Debit Card   
4                    Opening a Fixed Deposit Account   

                                         explanation  
0  The representative words and documents center ...  
1  The representative words and documents highlig...  
2  The representative words and documents suggest...  
3  The representative words and documents focus o...  
4  The representative words focus on terms relate...

# Sentiment Classification using Representative_Docs

## TBC

# Save to Disk

In [61]:
topic_info.to_csv("output_data/BERTopic_all_topic_info.csv", index=False, encoding='utf-8-sig')
topic_info.to_parquet("output_data/BERTopic_all_topic_info.parquet", index=False)

topics_df.to_csv(f"output_data/BERTopic_top_{top_N}_topic_info.csv", index=False, encoding='utf-8-sig')
topics_df.to_parquet(f"output_data/BERTopic_top_{top_N}_topic_info.parquet", index=False)

# Save to Topics Database

### TBC